In [1]:
from Bio import SeqIO
from collections import defaultdict, Counter
import sys

# Load Reference Genome 
def load_reference_genome(file_path):
    for record in SeqIO.parse(file_path, "fasta"):
        return str(record.seq)

# Load Paired Reads
def load_paired_reads(file_path):
    paired_reads = []
    current_pair = {}

    for record in SeqIO.parse(file_path, "fasta"):
        read_id = record.id
        sequence = str(record.seq)

        if read_id.endswith("/1"):
            read_num = 1
        elif read_id.endswith("/2"):
            read_num = 2
        else:
            continue

        base_id = read_id.rsplit("/", 1)[0]

        if base_id not in current_pair:
            current_pair[base_id] = {}

        current_pair[base_id][read_num] = sequence

        if 1 in current_pair[base_id] and 2 in current_pair[base_id]:
            paired_reads.append((current_pair[base_id][1], current_pair[base_id][2]))
            del current_pair[base_id]

    return paired_reads

# K-mer Indexing
def build_kmer_index(reference, k=15):
    index = defaultdict(list)
    for i in range(len(reference) - k + 1):
        kmer = reference[i:i + k]
        index[kmer].append(i)
    return index

# Align Read to Reference with Hamming Distance
def hamming_distance(s1, s2):
    if len(s1) != len(s2): return sys.maxsize
    return sum(c1 != c2 for c1, c2 in zip(s1, s2))

def map_read_to_reference(read, reference, kmer_index, k=15, max_mismatches=5):
    best_pos = None
    best_score = sys.maxsize

    for i in range(len(read) - k + 1):
        kmer = read[i:i + k]
        if kmer in kmer_index:
            for ref_pos in kmer_index[kmer]:
                start = ref_pos - i
                end = start + len(read)
                if start < 0 or end > len(reference): continue
                ref_sub = reference[start:end]
                dist = hamming_distance(read, ref_sub)
                if dist < best_score and dist <= max_mismatches:
                    best_score = dist
                    best_pos = start
    return best_pos

# Step 5: Call Substitutions 
def call_substitutions(reference, paired_reads, kmer_index, k=15):
    sub_counts = defaultdict(Counter)

    for read1, read2 in paired_reads:
        for read in [read1, read2]:
            pos = map_read_to_reference(read, reference, kmer_index, k)
            if pos is None: continue
            for i, base in enumerate(read):
                ref_index = pos + i
                if ref_index < len(reference):
                    ref_base = reference[ref_index]
                    if base != ref_base:
                        sub_counts[ref_index][base] += 1

    # Filter with minimum read support threshold
    predictions = []
    for pos, counter in sub_counts.items():
        total = sum(counter.values())
        base, count = counter.most_common(1)[0]
        if count >= 3:  # threshold: at least 3 reads support the variant
            predictions.append((pos, reference[pos], base))

    return sorted(predictions, key=lambda x: x[0])

# Step 6: Save Substitutions to File
def save_predictions(predictions, output_file):
    with open(output_file, "w") as f:
        for pos, ref_base, alt_base in predictions:
            f.write(f">S{pos} {ref_base} {alt_base}\n")

# ---- Step 7: Main Pipeline ---- #
def main():
    ref_file = "project1a_reference_genome.fasta"
    reads_file = "project1a_with_error_paired_reads.fasta"
    output_file = "predicted_mutations_project1a.txt"

    print("Loading reference genome...")
    reference = load_reference_genome(ref_file)

    print("Loading paired reads...")
    paired_reads = load_paired_reads(reads_file)

    print("Building k-mer index...")
    kmer_index = build_kmer_index(reference, k=15)

    print("Calling substitutions...")
    substitutions = call_substitutions(reference, paired_reads, kmer_index, k=15)

    print(f"Found {len(substitutions)} substitution candidates.")
    save_predictions(substitutions, output_file)
    print(f"Predictions saved to {output_file}.")

if __name__ == "__main__":
    main()

Loading reference genome...
Loading paired reads...
Building k-mer index...
Calling substitutions...
Found 150 substitution candidates.
Predictions saved to predicted_mutations_project1a.txt.
